# 池化层
通常当我们处理图像时，我们希望逐渐降低隐藏表示的空间分辨率、聚集信息，这样随着我们在神经网络中层叠的上升，每个神经元对其其敏感的感受野就越大。另外，我们还希望在检测较底层的特征时可以保持某种程度上的平移不变性，即如果原始特征出现了微小的扰动，希望卷积结果不会发生较大改变。

池化层具有的作用：
* 减小参数量，降低模型复杂度，一定程度上降低过拟合
* 扩大单个神经元的感受野,汇聚信息
* 保持特征不变形

## 最大池化层和平均池化层

In [1]:
import torch
import torch.nn as nn

In [6]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w  = pool_size
    Y = torch.zeros((X.shape[0] - p_h +1, X.shape[1] - p_w +1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i:i+p_h, j:j+p_w].max()
            elif mode == 'mean':
                Y[i, j] = X[i:i+p_h, j:j+p_w].mean()
    return Y

In [9]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
X

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])

In [7]:
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [8]:
pool2d(X, (2,2), mode='mean')

tensor([[2., 3.],
        [5., 6.]])

## 池化层的填充和步幅
与卷积层一样，池化层也可以通过填充和步幅来改变输出形状，下面我们使用深度学习框架中的二维最大池化层，来演示池化层中填充和步幅的作用。

首先我们构造一个输入张量，它具有四个维度，批量大小、通道、高度‘宽度。’

In [10]:
X = torch.arange(16, dtype=torch.float32).reshape(1, 1, 4, 4)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

默认情况下，深度学习框架中的步幅与池化窗口的大小相同，因此，如果我们使用（3,3）的池化窗口，那么我们的步幅就是（3,3），比如：

In [12]:
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

但我们也可以手动设置填充和步幅：

In [13]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

当然，我们可以(**设定一个任意大小的矩形汇聚窗口，并分别设定填充和步幅的高度和宽度**)。

In [14]:
pool2d = nn.MaxPool2d((2, 3), padding=(0, 1), stride=(2, 3))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

## 多个通道
在处理多通道输入数据时，池化层在每个通道上单独运算，这意味着池化层的输出通道与输入通道数相同，这与卷积层不一样。

下面，我们将在通道维度上连接张量`X`和`X+1`，以构建具有两个通道的输入：

In [15]:
X = torch.cat((X, X+1), 1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [16]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

如上所示，汇聚后输出通道的数量仍然是2。